In [1]:
import os
os.chdir('C:/Users/rokka/GH-repos/idea-lab-sw-isax')

import glob
import datetime as dt
import numpy as np
from tqdm import tqdm

# import local libraries
import fdl21.data.prototyping_metrics as pm
import fdl21.data.generate_catalog as gc
from fdl21 import isax_model
from fdl21.experiments import run_isax_experiments_sf as isax_exp_sf

#catalog_fname = 'data/omni_master_catalog_1995_2019.csv'

ERROR [jupyter:notebook_metadata:224] Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jasmine-kobayashi (idea-lab). Use `wandb login --relogin` to force relogin


In [3]:
isax_exp_sf.run_experiment(input_file= None,                   #if None, cut of catalog is used
                           start_date= dt.datetime(2018, 1, 1),  # start date of catalog cut
                           stop_date= dt.datetime(2018, 1, 2),   # end date of catalog cut
                           cadence=dt.timedelta(seconds=60),
                           cache=True,
                           instrument='omni')

INFO [run_isax_experiments_sf:run_experiment:645] Found 1 between 2018-01-01 00:00:00 2018-01-02 00:00:00


Creating file caches and histograms...:   0%|          | 0/1 [00:00<?, ?it/s]INFO [isax_model:build_cache:865] Loading cached file...
c:\Users\rokka\anaconda3\envs\idea-isax-sw\Lib\site-packages\tslearn\piecewise\piecewise.py:152: RuntimeWarning: Mean of empty slice.
  X_transformed[i_ts, i_seg, :] = segment.mean(axis=0)
c:\Users\rokka\anaconda3\envs\idea-isax-sw\Lib\site-packages\numpy\core\_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
c:\Users\rokka\anaconda3\envs\idea-isax-sw\Lib\site-packages\tslearn\piecewise\piecewise.py:152: RuntimeWarning: Mean of empty slice.
  X_transformed[i_ts, i_seg, :] = segment.mean(axis=0)
c:\Users\rokka\anaconda3\envs\idea-isax-sw\Lib\site-packages\numpy\core\_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
c:\Users\rokka\anaconda3\envs\idea-isax-sw\Lib\site-packages\tslearn\piecewise\piecewise.py:152: RuntimeWarning: Mean of empty slice.
  X_transformed[i_ts, i_

Bad Files------------------------------------------------------------
['/2018/omni_hro_1min_20180101_v01.cdf']
------------------------------------------------------------


Problem at: c:\Users\rokka\anaconda3\envs\idea-isax-sw\Lib\site-packages\wandb\sdk\wandb_init.py 837 getcaller


type: It appears that you do not have permission to access the requested resource. Please reach out to the project owner to grant you access. If you have the correct permissions, verify that there are no issues with your networking setup.(Error 403: Forbidden)